## Imports

In [2]:
from pathlib import Path
import numpy as np
import shutil
import xarray as xr

from flood_adapt import FloodAdapt
from flood_adapt.misc.exceptions import AlreadyExistsError
from flood_adapt.objects import (
    EventSet,
    Event,
    Projection, 
    PhysicalProjection, 
    SocioEconomicChange, 
    Scenario, 
    Strategy,
    FloodProof,
    SelectionType,
    )

from flood_adapt.objects.forcing.unit_system import UnitTypesLength, UnitfulLength, UnitfulLengthRefValue, VerticalReference
from flood_adapt.config.config import Settings


## Step 1a: Inputs (Event set and SLR)

Provide the event set and the range of SLR values to be calculated. 

In [3]:
DATA_DIR = Path(r"c:\Users\winter_ga\Offline_data\FloodAdapt-WorkingDatabase\Charleston\4_FloodAdapt\Database")
site = "charleston_beta_release"
name_event_set = "event_set_coastal"

slr = np.arange(0, 3.1, 1)
unit = UnitTypesLength.feet
fp_height = 2  # floodproof height 
timestep = 1  # timestep of Monte Carlo in years


## Step 1b: Reading FloodAdapt database and event set
Copy subevents to event folder

In [4]:
# Setup FloodAdapt
fn_event_set = DATA_DIR / site / "input" / "events" / name_event_set / f"{name_event_set}.toml"
settings = Settings(
    DATABASE_ROOT=DATA_DIR,
    DATABASE_NAME=site,
    SFINCS_BIN_PATH=Path(r"c:\Users\winter_ga\Offline_data\FloodAdapt-Database\system\win-64\sfincs_v.2.1_alpha\sfincs.exe"),
    FIAT_BIN_PATH=Path(r"c:\Users\winter_ga\Offline_data\FloodAdapt-Database\system\win-64\fiat_v0.2.1\fiat.exe"),
    VALIDATE_BINARIES=True,
)
fa = FloodAdapt(database_path=settings.database_path
                )

event_set = EventSet.load_file(fn_event_set)
events = []

# Copy sub_events directories to events folder
eventset_folder = DATA_DIR / site / "input" / "events"
for sub in event_set.sub_events:
    if sub.frequency <= timestep:
        events.append(sub.name)
        src_dir = DATA_DIR / site / "input" / "events" / name_event_set / sub.name
        dst_dir = DATA_DIR / site / "input" / "events" / sub.name
        
        if src_dir.exists() and src_dir.is_dir():
            shutil.copytree(src_dir, dst_dir, dirs_exist_ok=True)
            print(f"Copied {sub.name}")
        else:
            print(f"Source {src_dir} does not exist")


2025-11-22 02:37:00 PM - FloodAdapt.Database - INFO - Initializing database to charleston_beta_release at c:\users\winter_ga\offline_data\floodadapt-workingdatabase\charleston\4_floodadapt\database
2025-11-22 02:37:09 PM - FloodAdapt.FiatAdapter - INFO - Deleted Delft-FIAT simulation folder: c:\Users\winter_ga\Offline_data\FloodAdapt-WorkingDatabase\Charleston\4_FloodAdapt\Database\charleston_beta_release\output\scenarios\SLR_0_subevent_2_floodproof_all\Impacts\fiat_model
2025-11-22 02:37:09 PM - FloodAdapt.SfincsAdapter - INFO - Deleted simulation folder: c:\Users\winter_ga\Offline_data\FloodAdapt-WorkingDatabase\Charleston\4_FloodAdapt\Database\charleston_beta_release\output\scenarios\SLR_0_subevent_2_no_measures\Flooding\simulations\overland
2025-11-22 02:37:09 PM - FloodAdapt.FiatAdapter - INFO - Deleted Delft-FIAT simulation folder: c:\Users\winter_ga\Offline_data\FloodAdapt-WorkingDatabase\Charleston\4_FloodAdapt\Database\charleston_beta_release\output\scenarios\SLR_0_subevent_2_

Copied subevent_2
Copied subevent_3
Copied subevent_4


## Step 2a: Create projections

In [5]:
# create projections for all SLR values
projections = []
se_change = SocioEconomicChange(population_growth_existing=0,
                          economic_growth=0)
for nn,s in enumerate(slr):
    phys_proj = PhysicalProjection(sea_level_rise=UnitfulLength(value=s, units=unit))
    proj = Projection(
        name=f"SLR_{nn}",
        description=f"Sea level rise of {s:.2f} in {unit}",
        physical_projection=phys_proj, 
        socio_economic_change=se_change
    )
    
    try:
        fa.save_projection(proj)
    except AlreadyExistsError:
        print(f"Projection {proj.name} already exists in database")
    projections.append(proj)

projections

Projection SLR_0 already exists in database
Projection SLR_1 already exists in database
Projection SLR_2 already exists in database
Projection SLR_3 already exists in database


[Projection(name='SLR_0', description='Sea level rise of 0.00 in UnitTypesLength.feet', physical_projection=PhysicalProjection(sea_level_rise=UnitfulLength(value=0.0, units=UnitTypesLength.feet), subsidence=UnitfulLength(value=0.0, units=UnitTypesLength.meters), rainfall_multiplier=1.0, storm_frequency_increase=0.0), socio_economic_change=SocioEconomicChange(population_growth_existing=0.0, economic_growth=0.0, population_growth_new=0.0, new_development_elevation=None, new_development_shapefile=None)),
 Projection(name='SLR_1', description='Sea level rise of 1.00 in UnitTypesLength.feet', physical_projection=PhysicalProjection(sea_level_rise=UnitfulLength(value=1.0, units=UnitTypesLength.feet), subsidence=UnitfulLength(value=0.0, units=UnitTypesLength.meters), rainfall_multiplier=1.0, storm_frequency_increase=0.0), socio_economic_change=SocioEconomicChange(population_growth_existing=0.0, economic_growth=0.0, population_growth_new=0.0, new_development_elevation=None, new_development_shap

## Step 2b: Create strategy

In [6]:
# Create one strategy without measures and one with floodproofing all buildings

flood_proof = FloodProof(
    name="floodproof_all",
    description="Floodproofing all buildings.",
    selection_type=SelectionType.all,
    property_type="ALL",
    elevation=UnitfulLength(value=fp_height, units=unit)
)
try: 
    fa.save_measure(flood_proof)
except AlreadyExistsError:
    print("Strategy already exists in database")

strategies = [
    Strategy(
        name="no_measures",
        description="No measures",
        measures=[],
    ),
    Strategy(
        name="floodproof_all",
        description="Floodproof all buildings",
        measures=[flood_proof.name],
    )
]

for strat in strategies:
    try: 
        fa.save_strategy(strat)
    except AlreadyExistsError:
        print("Strategy already exists in database")

strategies

Strategy already exists in database
Strategy already exists in database
Strategy already exists in database


[Strategy(name='no_measures', description='No measures', measures=[]),
 Strategy(name='floodproof_all', description='Floodproof all buildings', measures=['floodproof_all'])]

## Step 3: Set up scenarios 

In [7]:
# events = event_set.load_sub_events(event_set.sub_events)
# print(event_set.sub_events)
scenarios = []
for strat in strategies:
    for proj in projections:
        for event_name in events:
            scen = Scenario(
                name=f"{proj.name}_{event_name}_{strat.name}",
                description=f"Scenario with {proj.description}, Event {event_name}, and {strat.description}",
                event=event_name,
                projection=proj.name,
                strategy=strat.name,
            )
            try: 
                fa.save_scenario(scen)
            except AlreadyExistsError:
                print("Scenario already exists in database")
            scenarios.append(scen)

Scenario already exists in database
Scenario already exists in database
Scenario already exists in database
Scenario already exists in database
Scenario already exists in database
Scenario already exists in database
Scenario already exists in database
Scenario already exists in database
Scenario already exists in database
Scenario already exists in database
Scenario already exists in database
Scenario already exists in database
Scenario already exists in database
Scenario already exists in database
Scenario already exists in database
Scenario already exists in database
Scenario already exists in database
Scenario already exists in database
Scenario already exists in database
Scenario already exists in database
Scenario already exists in database
Scenario already exists in database
Scenario already exists in database
Scenario already exists in database


## Step 4: Run scenarios

In [8]:
# for scen in scenarios[1:]:
#     print(scen.name)
#     fa.run_scenario(scen.name)

## Step 5: Arrange results in lookup dataset
dims: 
- object_id
- slr
- strategy

vars:
- water depth
- total damage

In [ ]:
import pandas as pd

scen_name=f"{projections[0].name}_{events[0]}_{strategies[0].name}"
gdf_temp = fa.get_building_footprint_impacts(scen_name)

ds_impacts = xr.Dataset(coords={
        "object_id": pd.to_numeric(gdf_temp["Object ID"], errors="coerce").astype("Int64"),
        "slr": slr, 
        "strategy": [s.name for s in strategies]
        },
        data_vars={
            "inun_depth": (
                ["object_id", "slr", "strategy"], 
                np.empty([len(gdf_temp["Object ID"]), len(slr), len(strategies)])
            ),
            "total_damage": (
                ["object_id", "slr", "strategy"],
                np.empty([len(gdf_temp["Object ID"]), len(slr), len(strategies)])
            )
        }
)
ds_impacts

for strat in strategies:
    for proj in projections:
        for event_name in events:
            scen_name=f"{proj.name}_{event_name}_{strat.name}"
            gdf_impacts = fa.get_building_footprint_impacts(scen_name)
            ds_impacts["inun_depth"].loc[:, proj.physical_projection.sea_level_rise.value, strat.name] = gdf_impacts["Inundation Depth"].values
            ds_impacts["total_damage"].loc[:, proj.physical_projection.sea_level_rise.value, strat.name] = gdf_impacts["Total Damage"].values





In [ ]:
ds_impacts["total_damage"].isel(slr=0,strategy=0).sum()

<xarray.DataArray 'total_damage' ()> Size: 8B
array(5640824.)
Coordinates:
    slr       float64 8B 0.0
    strategy  <U14 56B 'no_measures'